# 🤖 Citadel Agent Frameworks Testing Center

## Test Access Contracts with Different Agent Frameworks!

This notebook tests the Citadel Access Contracts using different agent frameworks to simulate multi-turn conversations:

| Access Contract | Agent Framework | Integration Type |
|----------------|-----------------|------------------|
| **Sales-Assistant** | Microsoft Agent Framework | Azure Key Vault (endpoint + key) |
| **HR-ChatAgent** | Microsoft Foundry Agent SDK | Foundry Project Connection |
| **Support-Bot** | LangChain | Local (direct endpoint + key) |

Each agent will simulate a conversation with a user around the topic of its access contract title.

> **Prerequisites:**
> - Citadel Governance Hub deployed with access contracts
> - Run `citadel-access-contracts-tests.ipynb` first to create the test contracts
> - Python packages: `agent-framework`, `azure-ai-projects`, `langchain`, `langchain-openai` among others (see step )
> - Have both Azure Key Vault and Foundry target resources provisioned and accessible (permissions and network)

<a id='0'></a>
### 0️⃣ Initialize Notebook Variables

Configure the same environment variables used in the access contracts tests.

In [ ]:
import os
import sys
import json
import requests
import time
import asyncio
import nest_asyncio

# Allow nested asyncio event loops (required for Jupyter)
nest_asyncio.apply()

sys.path.insert(1, '../shared')  # add the shared directory to the Python path
import utils
from apimtools import APIMClientTool

inference_api_version = "2024-05-01-preview"
targetInferenceApi = "models"  # use 'models' for universal LLM API, or 'openai' for Azure OpenAI

governance_hub_resource_group = "rg-ai-hub-citadel-dev-34"  ## specify the resource group name
location = "swedencentral"  ## specify the location of the Governance Hub

# Azure Key Vault configuration (for Sales-Assistant - Microsoft Agent Framework)
keyvault_subscription_id = "d2e7f84f-2790-4baa-9520-59ae8169ed0d"
keyvault_resource_group = "rg-foundry-agent-spoke-01"
keyvault_name = "kv-foundry-spoke-01"

# Azure AI Foundry configuration (for HR-ChatAgent - Foundry Agent SDK)
use_foundry_integration = True
foundry_subscription_id = "d2e7f84f-2790-4baa-9520-59ae8169ed0d"
foundry_resource_group = "rg-foundry-agent-spoke-01"
foundry_account_name = "msf-foundry-agent-spoke-01"
foundry_project_name = "crm-support-agent"
foundry_connection_name = "HR-ChatAgent-DEV-LLM"  # Connection created by access contract

# Model configuration
model_name = "gpt-4o"  # Model to use for agents

# Retry configuration
MAX_RETRIES = 3          # Maximum number of retries per call
RETRY_DELAY_BASE = 2     # Base delay in seconds (exponential backoff: base * 2^attempt)

# Store metrics for each agent (including retries)
agent_metrics = {
    "MS Agent Framework (Sales)": {"total_tokens": 0, "prompt_tokens": 0, "completion_tokens": 0, "calls": 0, "retries": 0},
    "Foundry SDK (HR)": {"total_tokens": 0, "prompt_tokens": 0, "completion_tokens": 0, "calls": 0, "retries": 0},
    "LangChain (Support)": {"total_tokens": 0, "prompt_tokens": 0, "completion_tokens": 0, "calls": 0, "retries": 0}
}

utils.print_ok("Notebook variables initialized!")

<a id='1'></a>
### 1️⃣ Verify Azure CLI and Initialize APIM Client

Connect to Azure and discover the deployed access contracts.

In [ ]:
output = utils.run("az account show", "Retrieved az account", "Failed to get the current az account")

if output.success and output.json_data:
    current_user = output.json_data['user']['name']
    tenant_id = output.json_data['tenantId']
    subscription_id = output.json_data['id']

    utils.print_info(f"Current user: {current_user}")
    utils.print_info(f"Tenant ID: {tenant_id}")
    utils.print_info(f"Subscription ID: {subscription_id}")

In [ ]:
try:
    apimClientTool = APIMClientTool(governance_hub_resource_group)
    apimClientTool.initialize()
    apimClientTool.discover_api(targetInferenceApi)

    apim_resource_gateway_url = str(apimClientTool.apim_resource_gateway_url)
    azure_endpoint = str(apimClientTool.azure_endpoint)
    
    # Get supported models
    supported_models = apimClientTool.get_policy_fragment_supported_models("set-backend-pools")
    utils.print_info(f"Supported models: {supported_models}")

    if targetInferenceApi == "openai":
        chat_completions_url = f"{azure_endpoint}openai/deployments/{{model_name}}/chat/completions?api-version={inference_api_version}"
    else:
        chat_completions_url = f"{azure_endpoint}models/chat/completions?api-version={inference_api_version}"
    
    utils.print_info(f"Chat Completion Endpoint: {chat_completions_url}")
    utils.print_ok(f"APIM Client initialized!")
except Exception as e:
    utils.print_error(f"Error initializing APIM Client: {e}")

<a id='2'></a>
### 2️⃣ Retrieve Access Contract API Keys

Get the API keys for each access contract to test with different agent frameworks.

In [ ]:
# Define the access contracts to test (matching those from citadel-access-contracts-tests.ipynb)
access_contract_configs = [
    {
        "name": "Sales-Assistant",
        "business_unit": "Sales",
        "use_case_name": "Assistant",
        "environment": "DEV",
        "agent_framework": "Microsoft Agent Framework",
        "endpoint_secret": "SALES-LLM-ENDPOINT",  # Key Vault secret names from access contract
        "apikey_secret": "SALES-LLM-KEY",
        "description": "Sales Assistant - Helps with sales inquiries, product information, and pricing"
    },
    {
        "name": "HR-ChatAgent",
        "business_unit": "HR",
        "use_case_name": "ChatAgent",
        "environment": "DEV",
        "agent_framework": "Foundry SDK",
        "foundry_connection": foundry_connection_name,  # Foundry connection created by access contract
        "description": "HR Chat Agent - Assists with HR policies, benefits, and employee questions"
    },
    {
        "name": "Support-Bot",
        "business_unit": "Support",
        "use_case_name": "Bot",
        "environment": "DEV",
        "agent_framework": "LangChain",
        "description": "Support Bot - Provides technical support and troubleshooting assistance"
    }
]

# Map contracts to their subscription keys
contract_keys = {}

for config in access_contract_configs:
    product_id = f"LLM-{config['business_unit']}-{config['use_case_name']}-{config['environment']}"
    subscription_name = f"{product_id}-SUB-01"
    
    for sub in apimClientTool.apim_subscriptions:
        if subscription_name.lower() in sub.get('name', '').lower():
            contract_keys[config['name']] = {
                "key": sub.get('key'),
                "product_id": product_id,
                "agent_framework": config['agent_framework'],
                "description": config['description'],
                "config": config  # Store full config for later use
            }
            utils.print_ok(f"Found key for {config['name']} ({config['agent_framework']})")
            break
    else:
        utils.print_warning(f"No key found for {config['name']} - run citadel-access-contracts-tests.ipynb first")

utils.print_info(f"\nRetrieved keys for {len(contract_keys)} access contracts")

<a id='3'></a>
### 3️⃣ Install Required Agent Frameworks

Install the necessary Python packages for each agent framework.

In [ ]:
# Install agent framework packages
packages = [
    "agent-framework",                 # Microsoft Agent Framework
    "azure-ai-projects>=2.0.0b2",     # Microsoft Foundry Agent SDK
    "langchain>=0.2.0",               # LangChain
    "langchain-openai>=0.1.0",        # LangChain OpenAI integration
    "azure-identity",                  # Azure authentication
    "azure-keyvault-secrets",          # Azure Key Vault for Sales-Assistant
    "nest-asyncio",                    # For running async in Jupyter
    "pywin32",                         # For Windows users to avoid event loop issues
    "matplotlib"                       # For visualizations
]

utils.print_info("Installing agent framework packages...")
for package in packages:
    try:
        %pip install -q {package}
        utils.print_ok(f"Installed {package}")
    except Exception as e:
        utils.print_warning(f"Could not install {package}: {e}")

utils.print_ok("Package installation complete!")

---
## 🧠 Agent Framework Implementations
---

<a id='4'></a>
### 4️⃣ Microsoft Agent Framework (Sales-Assistant)

Create a Sales Assistant agent using **Microsoft Agent Framework**.
This agent retrieves its endpoint and API key from **Azure Key Vault** based on the access contract parameters.

> **Key Vault Secrets:**
> - `SALES-LLM-ENDPOINT`: The LLM endpoint URL
> - `SALES-LLM-KEY`: The API key for authentication

In [ ]:
from azure.identity import DefaultAzureCredential
from azure.keyvault.secrets import SecretClient
from datetime import datetime

class MSAgentFrameworkSalesAgent:
    """Sales Assistant Agent using Microsoft Agent Framework with Azure Key Vault integration.
    
    Uses ChatAgent with OpenAIChatClient configured with custom headers to work with 
    Azure API Management gateways that expect the 'api-key' header.
    
    Note: We use OpenAIChatClient (not AzureOpenAIChatClient) because:
    - OpenAIChatClient uses standard OpenAI URL pattern: {base_url}/chat/completions
    - AzureOpenAIChatClient uses Azure-specific pattern: {endpoint}/openai/deployments/{deployment}/...
    - APIM gateways typically expect the standard OpenAI pattern with api-key header
    """
    
    def __init__(self, keyvault_name: str, endpoint_secret_name: str, apikey_secret_name: str, model_name: str):
        self.keyvault_name = keyvault_name
        self.endpoint_secret_name = endpoint_secret_name
        self.apikey_secret_name = apikey_secret_name
        self.model_name = model_name
        self.conversation_history = []
        self.total_tokens = 0
        self.prompt_tokens = 0
        self.completion_tokens = 0
        self.calls = 0
        self.retries = 0
        self.agent = None
        self.thread = None  # AgentThread for conversation context
        
        # Retrieve endpoint and API key from Azure Key Vault
        self._init_from_keyvault()
        
        # System prompt for Sales Assistant
        self.system_prompt = f"""You are a Sales Assistant AI agent. Your role is to:
- Answer questions about products and services
- Provide pricing information and discounts
- Help customers find the right solutions for their needs
- Handle sales inquiries professionally and helpfully

Be concise, professional, and always try to guide customers toward a purchase decision.
Current date: {datetime.now().strftime("%Y-%m-%d")}"""
        
        # Initialize Microsoft Agent Framework agent
        self._init_agent()
    
    def _init_from_keyvault(self):
        """Retrieve endpoint and API key from Azure Key Vault."""
        try:
            credential = DefaultAzureCredential()
            keyvault_uri = f"https://{self.keyvault_name}.vault.azure.net"
            client = SecretClient(vault_url=keyvault_uri, credential=credential)
            
            # Get endpoint from Key Vault
            endpoint_secret = client.get_secret(self.endpoint_secret_name)
            self.endpoint = endpoint_secret.value
            utils.print_ok(f"Retrieved endpoint: ({self.endpoint}) from Key Vault secret: {self.endpoint_secret_name}")
            
            # Get API key from Key Vault
            apikey_secret = client.get_secret(self.apikey_secret_name)
            self.api_key = apikey_secret.value
            utils.print_ok(f"Retrieved API key ({self.api_key[:4]}****) from Key Vault secret: {self.apikey_secret_name}")
            
        except Exception as e:
            utils.print_error(f"Failed to retrieve secrets from Key Vault: {e}")
            raise
    
    def _init_agent(self):
        """Initialize the Microsoft Agent Framework agent using ChatAgent with OpenAIChatClient.
        
        Uses OpenAIChatClient with custom default_headers to send 'api-key' header
        that Azure API Management expects, while using standard OpenAI URL patterns.
        """
        try:
            from agent_framework import ChatAgent
            from agent_framework.openai import OpenAIChatClient
            
            # Create OpenAI Chat Client with custom headers for APIM authentication
            # OpenAIChatClient uses standard /chat/completions URL pattern (what APIM expects)
            # We pass api-key via default_headers since APIM requires this header
            chat_client = OpenAIChatClient(
                base_url=self.endpoint,      # APIM endpoint from Key Vault (e.g., https://apim.../models)
                api_key="placeholder",       # Required but we override with default_headers
                model_id=self.model_name,    # Model name
                default_headers={
                    "api-key": self.api_key  # APIM subscription key as api-key header
                }
            )
            
            # Create ChatAgent using Microsoft Agent Framework
            self.agent = ChatAgent(
                chat_client=chat_client,
                name="SalesAssistant",
                instructions=self.system_prompt,
            )
            
            # Create a conversation thread for multi-turn conversations
            self.thread = self.agent.get_new_thread()
            
            utils.print_ok("Microsoft Agent Framework ChatAgent initialized successfully (using OpenAIChatClient with api-key header)")
            
        except ImportError as e:
            error_msg = (
                f"❌ Failed to import Microsoft Agent Framework: {e}\n"
                "  → Fix: Install the package with: pip install agent-framework\n"
                "  → Ensure you have the correct version installed"
            )
            utils.print_error(error_msg)
            raise RuntimeError(error_msg) from e
        except Exception as e:
            error_msg = (
                f"❌ Failed to initialize Microsoft Agent Framework ChatAgent: {e}\n"
                "  → Verify the endpoint from Key Vault is correct and accessible\n"
                "  → Verify the API key from Key Vault is valid\n"
                "  → Check that the APIM gateway is running and the subscription is active\n"
                f"  → Endpoint: {self.endpoint}\n"
                f"  → Model: {self.model_name}"
            )
            utils.print_error(error_msg)
            raise RuntimeError(error_msg) from e
    
    async def chat_async(self, user_message: str) -> str:
        """Send a message and get a response using Microsoft Agent Framework with retry logic."""
        self.conversation_history.append({"role": "user", "content": user_message})
        
        last_error = None
        for attempt in range(MAX_RETRIES + 1):
            try:
                # Use Microsoft Agent Framework with thread for conversation context
                result = await self.agent.run(user_message, thread=self.thread)
                content = result.text if hasattr(result, 'text') else str(result)
                
                # Track metrics (Microsoft Agent Framework may not expose token counts directly)
                # Estimate tokens based on response length
                estimated_tokens = len(content.split()) * 1.3  # rough estimate
                self.completion_tokens += int(estimated_tokens)
                self.prompt_tokens += len(user_message.split())
                self.total_tokens = self.prompt_tokens + self.completion_tokens
                self.calls += 1
                
                if attempt > 0:
                    utils.print_info(f"  ✅ Succeeded after {attempt} retry(ies)")
                
                self.conversation_history.append({"role": "assistant", "content": content})
                return content
                
            except Exception as e:
                last_error = e
                if attempt < MAX_RETRIES:
                    self.retries += 1
                    delay = RETRY_DELAY_BASE * (2 ** attempt)
                    utils.print_warning(f"  ⚠️ Attempt {attempt + 1}/{MAX_RETRIES + 1} failed: {e}")
                    utils.print_info(f"  ⏳ Retrying in {delay}s...")
                    await asyncio.sleep(delay)
                else:
                    # All retries exhausted — report clear failure
                    error_msg = (
                        f"❌ Microsoft Agent Framework call failed after {MAX_RETRIES + 1} attempts.\n"
                        f"  Last error: {last_error}\n"
                        f"  → Check that the APIM gateway is healthy and not throttling requests\n"
                        f"  → Verify the Sales-Assistant access contract rate limits\n"
                        f"  → Ensure the model '{self.model_name}' is available in the backend pool\n"
                        f"  → Endpoint: {self.endpoint}"
                    )
                    utils.print_error(error_msg)
                    self.conversation_history.append({"role": "assistant", "content": f"[ERROR] {error_msg}"})
                    return f"[ERROR] {error_msg}"
    
    def chat(self, user_message: str) -> str:
        """Synchronous wrapper for chat_async."""
        return asyncio.get_event_loop().run_until_complete(self.chat_async(user_message))
    
    def get_metrics(self) -> dict:
        """Return token usage metrics including retries."""
        return {
            "total_tokens": self.total_tokens,
            "prompt_tokens": self.prompt_tokens,
            "completion_tokens": self.completion_tokens,
            "calls": self.calls,
            "retries": self.retries
        }

utils.print_ok("Microsoft Agent Framework Sales Agent class defined (using OpenAIChatClient with api-key header)!")

In [ ]:
# Test the Microsoft Agent Framework Sales Agent with Key Vault integration
if "Sales-Assistant" in contract_keys:
    config = contract_keys["Sales-Assistant"]["config"]
    
    utils.print_info("🤖 Starting Microsoft Agent Framework Sales Agent conversation...")
    utils.print_info("="*60)
    utils.print_info(f"📦 Retrieving credentials from Azure Key Vault: {keyvault_name}")
    
    try:
        sales_agent = MSAgentFrameworkSalesAgent(
            keyvault_name=keyvault_name,
            endpoint_secret_name=config.get("endpoint_secret", "SALES-LLM-ENDPOINT"),
            apikey_secret_name=config.get("apikey_secret", "SALES-LLM-KEY"),
            model_name=model_name
        )
        
        # Simulate a sales conversation
        sales_conversation = [
            "Hi, I'm looking for an enterprise AI solution for my company.",
            "What features do you offer for data analytics?",
            "How much does the enterprise plan cost?",
            "Do you offer any discounts for annual subscriptions?",
            "Can you help me set up a demo?"
        ]
        
        for i, user_msg in enumerate(sales_conversation, 1):
            print(f"\n👤 User ({i}/{len(sales_conversation)}): {user_msg}")
            response = sales_agent.chat(user_msg)
            print(f"🤖 Sales Agent: {response[:300]}{'...' if len(response) > 300 else ''}")
            time.sleep(1)  # Rate limiting
        
        # Store metrics
        agent_metrics["MS Agent Framework (Sales)"] = sales_agent.get_metrics()
        utils.print_ok(f"\n📊 Sales Agent Metrics: {agent_metrics['MS Agent Framework (Sales)']}")
        
    except Exception as e:
        utils.print_error(f"Failed to initialize Sales Agent: {e}")
        utils.print_info("Make sure Key Vault secrets are configured and you have access permissions.")
else:
    utils.print_warning("Sales-Assistant contract not found. Skipping Microsoft Agent Framework test.")

<a id='5'></a>
### 5️⃣ Microsoft Foundry Agent SDK (HR-ChatAgent)

Create an HR Chat Agent using the **Microsoft Foundry Agent SDK** with the official **Prompt Agent** pattern.

> **Foundry Integration (based on official sample):**
> - Uses `PromptAgentDefinition` to create a versioned agent
> - Uses `conversations` API for multi-turn dialogue
> - Uses `responses` API to get agent responses
> - Deployment pattern: `connection_name/model_name` routes through Citadel
> - Authenticates using Azure DefaultAzureCredential
>
> **Reference:** [sample_agent_basic.py](https://github.com/Azure/azure-sdk-for-python/blob/main/sdk/ai/azure-ai-projects/samples/agents/sample_agent_basic.py)

In [ ]:
from azure.identity import DefaultAzureCredential
from datetime import datetime

class FoundryHRAgent:
    """HR Chat Agent using Microsoft Foundry Agent SDK with Prompt Agent pattern.
    
    Uses the Azure AI Projects SDK following the official sample pattern:
    - Creates a versioned Prompt Agent using PromptAgentDefinition
    - Uses conversations API for multi-turn dialogue
    - Uses responses API for agent responses
    - Uses get_openai_client() as context manager per the official SDK pattern
    
    Based on: https://github.com/Azure/azure-sdk-for-python/blob/main/sdk/ai/azure-ai-projects/samples/agents/sample_agent_basic.py
    """
    
    def __init__(self, foundry_account_name: str, foundry_project_name: str, 
                 connection_name: str, model_name: str):
        self.foundry_account_name = foundry_account_name
        self.foundry_project_name = foundry_project_name
        self.connection_name = connection_name
        self.model_name = model_name
        
        # Deployment name pattern: connection_name/model_name
        # This routes through the Foundry connection to Citadel governance hub
        self.deployment_name = f"{connection_name}/{model_name}"
        
        # Metrics tracking
        self.total_tokens = 0
        self.prompt_tokens = 0
        self.completion_tokens = 0
        self.calls = 0
        self.retries = 0
        
        # SDK clients and agent references
        self.credential = None
        self.project_client = None
        self.agent = None
        self.conversation_id = None  # Track conversation ID for multi-turn
        
        # System prompt for HR Assistant
        self.system_prompt = f"""You are an HR Chat Agent AI assistant. Your role is to:
- Answer questions about company HR policies
- Explain employee benefits and compensation packages
- Help with onboarding and offboarding procedures
- Provide guidance on workplace conduct and compliance
- Assist with leave requests and time-off policies

Be empathetic, professional, and ensure you protect employee privacy.
Always recommend consulting with HR directly for sensitive matters.
Current date: {datetime.now().strftime("%Y-%m-%d")}"""
        
        # Initialize Foundry client and create agent
        self._init_foundry_client()
    
    def _init_foundry_client(self):
        """Initialize Azure AI Foundry project client and create Prompt Agent."""
        try:
            from azure.ai.projects import AIProjectClient
            from azure.ai.projects.models import PromptAgentDefinition
            
            # Build the endpoint for the Foundry project
            # Format: https://{account}.services.ai.azure.com/api/projects/{project}
            endpoint = f"https://{self.foundry_account_name}.services.ai.azure.com/api/projects/{self.foundry_project_name}"
            
            utils.print_info(f"Connecting to Foundry endpoint: {endpoint}")
            
            # Create credential and project client
            self.credential = DefaultAzureCredential()
            self.project_client = AIProjectClient(
                endpoint=endpoint,
                credential=self.credential
            )
            utils.print_ok(f"Foundry project client initialized for: {self.foundry_project_name}")
            
            # Create a versioned Prompt Agent using the official pattern
            self.agent = self.project_client.agents.create_version(
                agent_name="HR-ChatAgent",
                definition=PromptAgentDefinition(
                    model=self.deployment_name,  # connection_name/model_name routes through Citadel
                    instructions=self.system_prompt,
                ),
            )
            utils.print_ok(f"Agent created (id: {self.agent.id}, name: {self.agent.name}, version: {self.agent.version})")
            utils.print_info(f"Using deployment: {self.deployment_name}")
            
        except ImportError as e:
            error_msg = (
                f"❌ Import error - SDK version may not support Agents API: {e}\n"
                "  → Fix: Install the correct SDK version with: pip install azure-ai-projects>=2.0.0b2\n"
                "  → Ensure azure-ai-projects package is installed and up to date"
            )
            utils.print_error(error_msg)
            raise RuntimeError(error_msg) from e
        except Exception as e:
            error_msg = (
                f"❌ Failed to initialize Foundry agent: {e}\n"
                f"  → Verify the Foundry account name: {self.foundry_account_name}\n"
                f"  → Verify the Foundry project name: {self.foundry_project_name}\n"
                f"  → Ensure the Foundry connection '{self.connection_name}' was created by the access contract\n"
                f"  → Check that DefaultAzureCredential has access to the Foundry project\n"
                f"  → Run 'az login' and ensure the correct subscription is selected"
            )
            utils.print_error(error_msg)
            raise RuntimeError(error_msg) from e
    
    def chat(self, user_message: str) -> str:
        """Send a message and get a response using Foundry Agents SDK with retry logic.
        
        Uses the conversations and responses API pattern from the official sample.
        Uses get_openai_client() as a context manager per the SDK pattern.
        """
        last_error = None
        for attempt in range(MAX_RETRIES + 1):
            try:
                return self._chat_with_agent(user_message)
            except Exception as e:
                last_error = e
                if attempt < MAX_RETRIES:
                    self.retries += 1
                    delay = RETRY_DELAY_BASE * (2 ** attempt)
                    utils.print_warning(f"  ⚠️ Attempt {attempt + 1}/{MAX_RETRIES + 1} failed: {e}")
                    utils.print_info(f"  ⏳ Retrying in {delay}s...")
                    time.sleep(delay)
                else:
                    error_msg = (
                        f"❌ Foundry Agent call failed after {MAX_RETRIES + 1} attempts.\n"
                        f"  Last error: {last_error}\n"
                        f"  → Check that the Foundry project '{self.foundry_project_name}' is accessible\n"
                        f"  → Verify the connection '{self.connection_name}' routes correctly to Citadel\n"
                        f"  → Ensure the model '{self.model_name}' is available via the connection\n"
                        f"  → Check the HR-ChatAgent access contract rate limits and quotas\n"
                        f"  → Deployment: {self.deployment_name}"
                    )
                    utils.print_error(error_msg)
                    return f"[ERROR] {error_msg}"
    
    def _chat_with_agent(self, user_message: str) -> str:
        """Chat using the Prompt Agent with conversations and responses API.
        
        Uses get_openai_client() as context manager, matching the official SDK pattern
        from hr-chatagent-foundry-test notebook.
        """
        with self.project_client.get_openai_client() as openai_client:
            if self.conversation_id is None:
                # Create a new conversation with the first user message
                conversation = openai_client.conversations.create(
                    items=[{"type": "message", "role": "user", "content": user_message}],
                )
                self.conversation_id = conversation.id
                utils.print_info(f"Created conversation (id: {self.conversation_id})")
            else:
                # Add user message to existing conversation
                openai_client.conversations.items.create(
                    conversation_id=self.conversation_id,
                    items=[{"type": "message", "role": "user", "content": user_message}],
                )
            
            # Get response from the agent
            response = openai_client.responses.create(
                conversation=self.conversation_id,
                extra_body={"agent": {"name": self.agent.name, "type": "agent_reference"}},
                input="",
            )
            
            content = response.output_text if hasattr(response, 'output_text') else str(response)
            
            # Track metrics - estimate tokens since responses API may not return usage
            estimated_prompt_tokens = len(user_message.split())
            estimated_completion_tokens = int(len(content.split()) * 1.3)
            self.prompt_tokens += estimated_prompt_tokens
            self.completion_tokens += estimated_completion_tokens
            self.total_tokens = self.prompt_tokens + self.completion_tokens
            self.calls += 1
            
            return content
    
    def get_metrics(self) -> dict:
        """Return token usage metrics including retries."""
        return {
            "total_tokens": self.total_tokens,
            "prompt_tokens": self.prompt_tokens,
            "completion_tokens": self.completion_tokens,
            "calls": self.calls,
            "retries": self.retries
        }
    
    def close(self, delete_version: bool = False):
        """Clean up resources.
        
        Args:
            delete_version: If True, deletes the agent version from Foundry. Default is False.
        """
        if delete_version:
            try:
                if self.agent and self.project_client:
                    self.project_client.agents.delete_version(
                        agent_name=self.agent.name,
                        agent_version=self.agent.version
                    )
                    utils.print_ok(f"Agent version deleted (name: {self.agent.name}, version: {self.agent.version})")
            except Exception as e:
                utils.print_warning(f"Failed to delete agent version: {e}")
        else:
            if self.agent:
                utils.print_info(f"Agent version retained (name: {self.agent.name}, version: {self.agent.version}). Pass delete_version=True to remove it.")

utils.print_ok("Foundry HR Agent class defined (using Prompt Agent with conversations/responses API)!")

In [ ]:
# Test the Foundry SDK HR Agent with project connection
if "HR-ChatAgent" in contract_keys:
    config = contract_keys["HR-ChatAgent"]["config"]
    
    utils.print_info("🏢 Starting Microsoft Foundry SDK HR Agent conversation...")
    utils.print_info("="*60)
    utils.print_info(f"📦 Connecting to Foundry project: {foundry_project_name}")
    utils.print_info(f"🔗 Using connection: {config.get('foundry_connection', foundry_connection_name)}")
    
    try:
        hr_agent = FoundryHRAgent(
            foundry_account_name=foundry_account_name,
            foundry_project_name=foundry_project_name,
            connection_name=config.get("foundry_connection", foundry_connection_name),
            model_name=model_name
        )
        
        # Simulate an HR conversation
        hr_conversation = [
            "Hello, I'm a new employee. Can you tell me about the benefits package?",
            "What is the vacation policy? How many days do I get?",
            "How do I request time off for a medical appointment?",
            "What's the process for reporting workplace issues?",
            "Thank you for your help! One more question - when is the next open enrollment period?"
        ]
        
        for i, user_msg in enumerate(hr_conversation, 1):
            print(f"\n👤 User ({i}/{len(hr_conversation)}): {user_msg}")
            response = hr_agent.chat(user_msg)
            print(f"🤖 HR Agent: {response[:300]}{'...' if len(response) > 300 else ''}")
            time.sleep(1)  # Rate limiting
        
        # Store metrics
        agent_metrics["Foundry SDK (HR)"] = hr_agent.get_metrics()
        utils.print_ok(f"\n📊 HR Agent Metrics: {agent_metrics['Foundry SDK (HR)']}")
        
        # Clean up (set delete_version=True to remove the agent version from Foundry)
        hr_agent.close(delete_version=False)
        
    except Exception as e:
        utils.print_error(f"Failed to initialize HR Agent: {e}")
        utils.print_info("Make sure the Foundry connection is created by the access contract deployment.")
else:
    utils.print_warning("HR-ChatAgent contract not found. Skipping Foundry SDK test.")

<a id='6'></a>
### 6️⃣ LangChain Agent (Support-Bot)

Create a Support Bot agent using the **LangChain** framework.
This is a local agent that uses variables for the LLM endpoint, key, and model name.

> **Local Configuration:**
> - Endpoint: Retrieved from APIM client
> - API Key: Retrieved from APIM subscription
> - Model: Specified in request body

In [ ]:
from datetime import datetime

class LangChainSupportAgent:
    """Support Bot Agent using LangChain framework with local configuration."""
    
    def __init__(self, llm_endpoint: str, api_key: str, model_name: str):
        """
        Initialize LangChain Support Agent with direct endpoint configuration.
        
        Args:
            llm_endpoint: The LLM API endpoint URL
            api_key: The API key for authentication
            model_name: The model name to include in request body
        """
        self.llm_endpoint = llm_endpoint
        self.api_key = api_key
        self.model_name = model_name
        self.messages = []
        self.total_tokens = 0
        self.prompt_tokens = 0
        self.completion_tokens = 0
        self.calls = 0
        self.retries = 0
        
        # System prompt for Support Bot
        self.system_prompt = f"""You are a Technical Support Bot AI assistant. Your role is to:
- Help users troubleshoot technical issues
- Provide step-by-step guidance for common problems
- Explain technical concepts in simple terms
- Escalate complex issues to human support when needed
- Log and track support tickets

Be patient, thorough, and always verify that the user's issue is resolved.
Ask clarifying questions when needed to diagnose problems accurately.
Current date: {datetime.now().strftime("%Y-%m-%d")}"""
        
        self.messages.append({"role": "system", "content": self.system_prompt})
        
        utils.print_ok(f"LangChain Support Agent initialized")
        utils.print_info(f"Endpoint: {self.llm_endpoint}")
        utils.print_info(f"Model: {self.model_name}")
    
    def chat(self, user_message: str) -> str:
        """
        Send a message and get a response using LangChain patterns with retry logic.
        Uses direct API calls with endpoint, key, and model in body.
        """
        self.messages.append({"role": "user", "content": user_message})
        
        # LangChain-style payload with model in body
        payload = {
            "model": self.model_name,  # Model name in body as per LangChain pattern
            "messages": self.messages,
            "max_tokens": 500,
            "temperature": 0.7
        }
        
        last_error = None
        for attempt in range(MAX_RETRIES + 1):
            try:
                # Direct API call using configured endpoint and key
                response = requests.post(
                    self.llm_endpoint,
                    headers={
                        "api-key": self.api_key,
                        "Content-Type": "application/json"
                    },
                    json=payload,
                    timeout=60
                )
                
                if response.status_code == 200:
                    data = response.json()
                    content = data.get("choices", [{}])[0].get("message", {}).get("content", "")
                    
                    # Track token usage
                    usage = data.get("usage", {})
                    self.total_tokens += usage.get("total_tokens", 0)
                    self.prompt_tokens += usage.get("prompt_tokens", 0)
                    self.completion_tokens += usage.get("completion_tokens", 0)
                    self.calls += 1
                    
                    if attempt > 0:
                        utils.print_info(f"  ✅ Succeeded after {attempt} retry(ies)")
                    
                    self.messages.append({"role": "assistant", "content": content})
                    return content
                elif response.status_code == 429:
                    # Rate limited — retry with backoff
                    retry_after = int(response.headers.get("Retry-After", RETRY_DELAY_BASE * (2 ** attempt)))
                    raise Exception(f"Rate limited (429). Retry-After: {retry_after}s. Body: {response.text[:200]}")
                else:
                    raise Exception(f"HTTP {response.status_code}: {response.text[:200]}")
                    
            except Exception as e:
                last_error = e
                if attempt < MAX_RETRIES:
                    self.retries += 1
                    delay = RETRY_DELAY_BASE * (2 ** attempt)
                    utils.print_warning(f"  ⚠️ Attempt {attempt + 1}/{MAX_RETRIES + 1} failed: {e}")
                    utils.print_info(f"  ⏳ Retrying in {delay}s...")
                    time.sleep(delay)
                else:
                    error_msg = (
                        f"❌ LangChain Support Agent call failed after {MAX_RETRIES + 1} attempts.\n"
                        f"  Last error: {last_error}\n"
                        f"  → Check that the APIM gateway is healthy and not throttling\n"
                        f"  → Verify the Support-Bot access contract rate limits and quotas\n"
                        f"  → Ensure the model '{self.model_name}' is available in the backend pool\n"
                        f"  → Endpoint: {self.llm_endpoint}"
                    )
                    utils.print_error(error_msg)
                    # Remove the user message that failed so conversation stays consistent
                    self.messages.pop()
                    return f"[ERROR] {error_msg}"
    
    def get_metrics(self) -> dict:
        """Return token usage metrics including retries."""
        return {
            "total_tokens": self.total_tokens,
            "prompt_tokens": self.prompt_tokens,
            "completion_tokens": self.completion_tokens,
            "calls": self.calls,
            "retries": self.retries
        }

utils.print_ok("LangChain Support Agent class defined!")

In [ ]:
# Test the LangChain Support Agent with local configuration
if "Support-Bot" in contract_keys:
    support_key = contract_keys["Support-Bot"]["key"]
    
    utils.print_info("🦜 Starting LangChain Support Agent conversation...")
    utils.print_info("="*60)
    utils.print_info(f"📦 Using local configuration with APIM endpoint")
    
    try:
        support_agent = LangChainSupportAgent(
            llm_endpoint=chat_completions_url,  # Local endpoint variable
            api_key=support_key,                 # Local API key variable
            model_name=model_name                # Model name in body
        )
        
        # Simulate a support conversation
        support_conversation = [
            "Hi, I'm having trouble connecting to the VPN. It keeps timing out.",
            "I'm using Windows 11 and the company VPN client version 3.2.",
            "I've tried restarting but the issue persists. What else can I try?",
            "The network adapter shows it's connected but no internet access through VPN.",
            "That worked! The DNS settings were incorrect. Thank you for your help!"
        ]
        
        for i, user_msg in enumerate(support_conversation, 1):
            print(f"\n👤 User ({i}/{len(support_conversation)}): {user_msg}")
            response = support_agent.chat(user_msg)
            print(f"🤖 Support Agent: {response[:300]}{'...' if len(response) > 300 else ''}")
            time.sleep(1)  # Rate limiting
        
        # Store metrics
        agent_metrics["LangChain (Support)"] = support_agent.get_metrics()
        utils.print_ok(f"\n📊 Support Agent Metrics: {agent_metrics['LangChain (Support)']}")
        
    except Exception as e:
        utils.print_error(f"Failed to initialize Support Agent: {e}")
else:
    utils.print_warning("Support-Bot contract not found. Skipping LangChain test.")

---
## 📊 Agent Performance Comparison
---

<a id='7'></a>
### 7️⃣ Display Agent Statistics and Comparison

Visualize token consumption across all agent frameworks using pie charts.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Filter agents with data
active_agents = {k: v for k, v in agent_metrics.items() if v.get('total_tokens', 0) > 0}

if active_agents:
    # Print summary table
    print("\n" + "="*90)
    print("📊 AGENT FRAMEWORKS PERFORMANCE SUMMARY")
    print("="*90)
    print(f"{'Agent Framework':<30} {'Calls':<8} {'Retries':<10} {'Prompt':<12} {'Completion':<12} {'Total':<12}")
    print("-"*90)
    
    total_all_tokens = 0
    total_all_retries = 0
    for agent_name, metrics in active_agents.items():
        retries = metrics.get('retries', 0)
        print(f"{agent_name:<30} {metrics['calls']:<8} {retries:<10} {metrics['prompt_tokens']:<12} {metrics['completion_tokens']:<12} {metrics['total_tokens']:<12}")
        total_all_tokens += metrics['total_tokens']
        total_all_retries += retries
    
    print("-"*90)
    print(f"{'TOTAL':<30} {'':<8} {total_all_retries:<10} {'':<12} {'':<12} {total_all_tokens:<12}")
    print("="*90)
    
    # Create visualization
    fig, axes = plt.subplots(1, 4, figsize=(20, 5))
    
    # Color scheme
    colors = ['#2E86AB', '#A23B72', '#F18F01']
    
    # 1. Total Tokens Pie Chart
    labels = list(active_agents.keys())
    total_tokens = [m['total_tokens'] for m in active_agents.values()]
    
    axes[0].pie(total_tokens, labels=labels, autopct='%1.1f%%', colors=colors[:len(labels)], 
                startangle=90, explode=[0.02]*len(labels))
    axes[0].set_title('Total Tokens by Agent', fontsize=12, fontweight='bold')
    
    # 2. Prompt vs Completion Tokens (Stacked Bar)
    x = np.arange(len(labels))
    prompt_tokens = [m['prompt_tokens'] for m in active_agents.values()]
    completion_tokens = [m['completion_tokens'] for m in active_agents.values()]
    
    axes[1].bar(x, prompt_tokens, label='Prompt Tokens', color='#2E86AB', alpha=0.8)
    axes[1].bar(x, completion_tokens, bottom=prompt_tokens, label='Completion Tokens', color='#F18F01', alpha=0.8)
    axes[1].set_xticks(x)
    axes[1].set_xticklabels([l.split('(')[0].strip() for l in labels], rotation=15, ha='right')
    axes[1].set_ylabel('Tokens')
    axes[1].set_title('Token Distribution by Agent', fontsize=12, fontweight='bold')
    axes[1].legend()
    
    # 3. API Calls vs Retries (Grouped Bar)
    calls = [m['calls'] for m in active_agents.values()]
    retries = [m.get('retries', 0) for m in active_agents.values()]
    
    bar_width = 0.35
    axes[2].bar(x - bar_width/2, calls, bar_width, label='Successful Calls', color='#2E86AB', alpha=0.8)
    axes[2].bar(x + bar_width/2, retries, bar_width, label='Retries', color='#E84855', alpha=0.8)
    axes[2].set_xticks(x)
    axes[2].set_xticklabels([l.split('(')[0].strip() for l in labels], rotation=15, ha='right')
    axes[2].set_ylabel('Count')
    axes[2].set_title('Calls vs Retries by Agent', fontsize=12, fontweight='bold')
    axes[2].legend()
    
    # Add value labels on bars
    for i, (call, retry) in enumerate(zip(calls, retries)):
        axes[2].text(i - bar_width/2, call + 0.1, str(call), ha='center', va='bottom', fontweight='bold', fontsize=9)
        if retry > 0:
            axes[2].text(i + bar_width/2, retry + 0.1, str(retry), ha='center', va='bottom', fontweight='bold', fontsize=9, color='#E84855')
    
    # 4. Retry Rate (% of total attempts that were retries)
    retry_rates = []
    for m in active_agents.values():
        total_attempts = m['calls'] + m.get('retries', 0)
        rate = (m.get('retries', 0) / total_attempts * 100) if total_attempts > 0 else 0
        retry_rates.append(rate)
    
    bar_colors = ['#4CAF50' if r == 0 else '#FF9800' if r < 30 else '#E84855' for r in retry_rates]
    bars = axes[3].bar(x, retry_rates, color=bar_colors, alpha=0.8)
    axes[3].set_xticks(x)
    axes[3].set_xticklabels([l.split('(')[0].strip() for l in labels], rotation=15, ha='right')
    axes[3].set_ylabel('Retry Rate (%)')
    axes[3].set_title('Retry Rate by Agent', fontsize=12, fontweight='bold')
    axes[3].set_ylim(0, max(max(retry_rates) * 1.3, 10))
    
    # Add value labels
    for bar, rate in zip(bars, retry_rates):
        axes[3].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.3, 
                     f'{rate:.1f}%', ha='center', va='bottom', fontweight='bold', fontsize=9)
    
    plt.tight_layout()
    plt.savefig('agent_metrics_comparison.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    utils.print_ok("Visualization saved to 'agent_metrics_comparison.png'")
else:
    utils.print_warning("No agent metrics available. Run the agent tests first.")

<a id='8'></a>
### 8️⃣ Token Efficiency Analysis

Analyze the token efficiency (completion tokens per call) for each agent.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

active_agents = {k: v for k, v in agent_metrics.items() if v.get('total_tokens', 0) > 0}

if active_agents:
    # Calculate efficiency metrics
    efficiency_data = []
    for agent_name, metrics in active_agents.items():
        calls = metrics['calls']
        retries = metrics.get('retries', 0)
        if calls > 0:
            avg_total = metrics['total_tokens'] / calls
            avg_prompt = metrics['prompt_tokens'] / calls
            avg_completion = metrics['completion_tokens'] / calls
            efficiency = metrics['completion_tokens'] / metrics['total_tokens'] * 100 if metrics['total_tokens'] > 0 else 0
            total_attempts = calls + retries
            reliability = (calls / total_attempts * 100) if total_attempts > 0 else 100.0
            
            efficiency_data.append({
                'name': agent_name,
                'avg_total': avg_total,
                'avg_prompt': avg_prompt,
                'avg_completion': avg_completion,
                'efficiency': efficiency,
                'retries': retries,
                'reliability': reliability
            })
    
    # Print efficiency table
    print("\n" + "="*95)
    print("📈 TOKEN EFFICIENCY & RELIABILITY ANALYSIS")
    print("="*95)
    print(f"{'Agent Framework':<30} {'Avg Total':<12} {'Avg Prompt':<12} {'Avg Compl.':<12} {'Efficiency':<12} {'Retries':<10} {'Reliability':<12}")
    print("-"*95)
    
    for data in efficiency_data:
        print(f"{data['name']:<30} {data['avg_total']:<12.1f} {data['avg_prompt']:<12.1f} {data['avg_completion']:<12.1f} {data['efficiency']:<11.1f}% {data['retries']:<10} {data['reliability']:<11.1f}%")
    
    print("="*95)
    print("\n💡 Efficiency = (Completion Tokens / Total Tokens) × 100")
    print("   Higher efficiency means more output for less input (context)")
    print("🔄 Reliability = (Successful Calls / Total Attempts) × 100")
    print("   Higher reliability means fewer retries needed")
    
    # Create side-by-side visualization: Efficiency vs Reliability
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
    
    labels = [d['name'].split('(')[0].strip() for d in efficiency_data]
    efficiencies = [d['efficiency'] for d in efficiency_data]
    reliabilities = [d['reliability'] for d in efficiency_data]
    retries_list = [d['retries'] for d in efficiency_data]
    colors = ['#2E86AB', '#A23B72', '#F18F01']
    
    # Left: Token Efficiency
    bars1 = ax1.barh(labels, efficiencies, color=colors[:len(labels)], alpha=0.8)
    for bar, eff in zip(bars1, efficiencies):
        ax1.text(bar.get_width() + 1, bar.get_y() + bar.get_height()/2, 
                f'{eff:.1f}%', va='center', fontweight='bold')
    ax1.set_xlabel('Efficiency (%)')
    ax1.set_title('Token Efficiency by Agent', fontsize=14, fontweight='bold')
    ax1.set_xlim(0, max(efficiencies) * 1.2)
    
    # Right: Reliability (with retry count annotation)
    reliability_colors = ['#4CAF50' if r == 100 else '#FF9800' if r > 80 else '#E84855' for r in reliabilities]
    bars2 = ax2.barh(labels, reliabilities, color=reliability_colors, alpha=0.8)
    for bar, rel, retries in zip(bars2, reliabilities, retries_list):
        label = f'{rel:.1f}%'
        if retries > 0:
            label += f' ({retries} retries)'
        ax2.text(bar.get_width() + 1, bar.get_y() + bar.get_height()/2, 
                label, va='center', fontweight='bold')
    ax2.set_xlabel('Reliability (%)')
    ax2.set_title('Call Reliability by Agent', fontsize=14, fontweight='bold')
    ax2.set_xlim(0, 120)
    ax2.axvline(x=100, color='#4CAF50', linestyle='--', alpha=0.3, label='Perfect reliability')
    ax2.legend(loc='lower right')
    
    plt.tight_layout()
    plt.show()
    
else:
    utils.print_warning("No agent metrics available for efficiency analysis.")

<a id='summary'></a>
### 📋 Summary

This notebook tested the following access contracts with different agent frameworks:

| Access Contract | Agent Framework | Use Case | Integration |
|----------------|-----------------|----------|-------------|
| Sales-Assistant | Microsoft Agent Framework | Sales inquiries & pricing | Azure Key Vault (endpoint + key) |
| HR-ChatAgent | Microsoft Foundry Agent SDK | HR policies & benefits | Foundry Project Connection |
| Support-Bot | LangChain | Technical support | Local (direct endpoint + key) |

Each agent simulated a multi-turn conversation relevant to its business domain.
The metrics collected help compare token consumption across different frameworks.

**Integration Patterns:**
- **Microsoft Agent Framework (Sales)**: Retrieves endpoint and API key from Azure Key Vault secrets, uses `AzureOpenAIChatClient.as_agent()`
- **Foundry SDK (HR)**: Uses `connection_name/model_name` deployment pattern via Foundry project
- **LangChain (Support)**: Local configuration with endpoint, key, and model in request body

In [ ]:
# Final summary
utils.print_info("\n" + "="*60)
utils.print_info("🎉 AGENT FRAMEWORKS TESTING COMPLETE!")
utils.print_info("="*60)

total_retries = 0
for agent_name, metrics in agent_metrics.items():
    retries = metrics.get('retries', 0)
    total_retries += retries
    if metrics.get('total_tokens', 0) > 0:
        retry_info = f", {retries} retries" if retries > 0 else ""
        utils.print_ok(f"✅ {agent_name}: {metrics['total_tokens']} total tokens, {metrics['calls']} calls{retry_info}")
    else:
        utils.print_warning(f"⚠️ {agent_name}: No data (contract may not be deployed)")

if total_retries > 0:
    utils.print_warning(f"\n🔄 Total retries across all agents: {total_retries}")
    utils.print_info("  Consider reviewing rate limits or backend pool capacity if retries are high")

utils.print_info("\n📌 Next Steps:")
utils.print_info("  1. Review token consumption patterns and retry rates")
utils.print_info("  2. Adjust rate limits in access contracts if retries are frequent")
utils.print_info("  3. Implement production-ready error handling")
utils.print_info("  4. Add monitoring and alerting for each agent type")